**Import required libraries**

In [ ]:
# Import utils
import numpy as np
import pandas as pd
import time
import pickle
import pyreadr
import json
from pathlib import Path
import joblib as joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm


# Import D2D Multi-Period Inventory Control Library
import WeightedSAA
from WeightedSAA import RobustWeightedSAA
from WeightedSAA import WeightedSAA
from WeightedSAA import Evaluation

**Functions**

We first define a 'wrapper' function that iterates the experiment for a given SKU over differen cost parameter settins and lengths of the rolling look-ahead horizon tau.

If the parameter sampling is provided (either 'global' or 'local), the function uses the specified sampling strategy. Else, SAA is performed. If the multiplier 'e' for the uncertainty set threshold epsilon is provided, the function performs the robust extension.
 
Where: epsilon[t] = e *  in-sample standard deviation of the current product (SKU).

The function prepares and calls the experiment over t=1...T for each cost paramater setting and look-ahead horizon tau and then summarises the results including performance and performance meta inormation. It also saves the results in CSV format to the specified path and the function can also be used in parallel processing environments.

In [ ]:
def run_experiments(SKU, **kwargs):
    
    """
    
    Description ...
    
    
    Arguments:
    
        SKU: product (SKU) identifier
        sale_yearweek: Last sale_yearweek of training data
        T: Length T of the test horizon
        tau: List of lengths of rolling look-ahead horizons
        cost_params: dictionary/dictionary of dictionaries of cost parameters {'CR', 'K', 'u', 'h', 'b'}
        gurobi_params: dictionary of gurobi meta params {'LogToConsole', 'Threads', 'NonConvex' 
                                                         'PSDTol', 'MIPGap': 'obj_improvement', 
                                                         'obj_timeout_sec'}
        path: directory where results should be saved
        model_name: model name for the file to save results
        
    Optional arguments:
    
        sampling: sampling strategy (either 'global' or 'local'); performs SAA if not provided
        e: robust uncertainty set threshold multiplier; performs no robust extension if not provided
    

    """
  
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Print progress
    if kwargs['print_progress']: 
        print('SKU:', SKU)
    
    # Initialize results
    evaluate = Evaluation()
    results = pd.DataFrame()
   
    # For each cost param setting
    for cost_params in kwargs['cost_params'].values():
        
        # Print progress
        if kwargs['print_progress']: 
            print('... cost param setting:', cost_params)
    
        # For each rolling look-ahead horizon
        for tau in kwargs['tau']:
            
            # Print progress
            if kwargs['print_progress']: 
                print('...... look-ahead horizon:', tau)
    
            ## Weighted (Robust) SAA
            if 'sampling' in kwargs:
    
                ## Weighted Robust SAA  
                if 'e' in kwargs:

                    # Prepare data
                    data = evaluate.prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                    y, ids_train, ids_test, weights, epsilons = data
                    
                    # Create empty model
                    wsaamodel = RobustWeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = evaluate.run_experiment(y, ids_train, ids_test, tau, wsaamodel, weights=weights, epsilons=epsilons, **cost_params)

                ## Weighted SAA
                else: 
                    
                    # Prepare data
                    data = evaluate.prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'])
                    y, ids_train, ids_test, weights, _ = data
                    
                    # Create empty model
                    wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = evaluate.run_experiment(y, ids_train, ids_test, tau, wsaamodel, weights=weights, **cost_params)


            ## SAA
            else:
                
                # Prepare data
                data = evaluate.prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES)
                y, ids_train, ids_test, _, _ = data

                # Create empty model
                wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                # Run rolling horizon model over t=1...T
                result = evaluate.run_experiment(y, ids_train, ids_test, tau, wsaamodel, **cost_params)

            
            ## ToDo: ExPost
            
            # Store result
            meta = pd.DataFrame({

                'SKU': np.repeat(SKU,kwargs['T']),
                'n_periods': np.repeat(kwargs['T'],kwargs['T']),
                'tau': np.repeat(tau,kwargs['T']),
                'CR': np.repeat(cost_params['CR'],kwargs['T']),
                'LogToConsole': np.repeat(kwargs['gurobi_params']['LogToConsole'],kwargs['T']),
                'Threads': np.repeat(kwargs['gurobi_params']['Threads'],kwargs['T']),
                'NonConvex': np.repeat(kwargs['gurobi_params']['NonConvex'],kwargs['T']),
                'PSDTol': np.repeat(kwargs['gurobi_params']['PSDTol'],kwargs['T']),
                'MIPGap': np.repeat(kwargs['gurobi_params']['MIPGap'],kwargs['T']),
                'obj_improvement': np.repeat(kwargs['gurobi_params']['obj_improvement'],kwargs['T']),
                'obj_timeout_sec': np.repeat(kwargs['gurobi_params']['obj_timeout_sec'],kwargs['T']),
                'e': np.repeat(kwargs['e'],kwargs['T']) if 'e' in kwargs else np.repeat(0,kwargs['T']),
                'epsilon': [epsilon for epsilon in epsilons] if 'e' in kwargs else np.repeat(0,kwargs['T'])
            })

            result = pd.concat([meta, result], axis=1)

            # Store
            if not results.empty:
                results = results.append(result)   
            else:
                results = pd.DataFrame(result) 

    # Save result
    save_log = results.to_csv(
        path_or_buf=kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'.csv', 
        sep=',', index=False
    )
    
    
    # Time
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Print progress
    if kwargs['print_progress']: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    
    # Returns results 
    if (kwargs['return_results'] if 'return_results' in kwargs else False):
        return results
    
    # Returns a log
    else:
        return  {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

This is a context manager for parellel execution with the purpose of reporting progress. 

Credits: https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

**Experiments**

In [ ]:
# Set folder names as global variables
os.chdir('/home/fesc/')
global PATH_DATA, PATH_PARAMS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

For the models specified, the code below runs the experiment for all given products (SKUs) and over parameter settings (e.g., cost parameters, horizon parameters, etc.). In total, we have 460 products (SKUs) with each 3 different cost parameter settings varying the critical ratio (CR) of holding and backlogging cost being {CR=0.50, CR=0.75, CR=0.90) and each 5 different lengths of the rolling look-ahead horizon tau being {1,2,3,4,5}.

**(a) Rolling Horizon Global Weighted SAA**

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAA',
    'model_name': 'GwSAA',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])
       
# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

**(b) Rolling Horizon Global Robust Weighted SAA**

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Robust uncertainty set threshold multiplier
    'e': 12,

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAAR',
    'model_name': 'GwSAAR',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])
       
# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
print(resultslog)

In [ ]:
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################

In [1]:
# Import utils
import numpy as np
import pandas as pd
import time
import pickle
import pyreadr
import json
from pathlib import Path
import joblib as joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm


# Import D2D Multi-Period Inventory Control Library
import WeightedSAA
from WeightedSAA import RobustWeightedSAA
from WeightedSAA import WeightedSAA
from WeightedSAA import Evaluation
from WeightedSAA import RobustWeightedSAA2

In [2]:
def run_experiments2(SKU, **kwargs):
    
    """
    
    Description ...
    
    
    Arguments:
    
        SKU: product (SKU) identifier
        sale_yearweek: Last sale_yearweek of training data
        T: Length T of the test horizon
        tau: List of lengths of rolling look-ahead horizons
        cost_params: dictionary/dictionary of dictionaries of cost parameters {'CR', 'K', 'u', 'h', 'b'}
        gurobi_params: dictionary of gurobi meta params {'LogToConsole', 'Threads', 'NonConvex' 
                                                         'PSDTol', 'MIPGap': 'obj_improvement', 
                                                         'obj_timeout_sec'}
        path: directory where results should be saved
        model_name: model name for the file to save results
        
    Optional arguments:
    
        sampling: sampling strategy (either 'global' or 'local'); performs SAA if not provided
        e: robust uncertainty set threshold multiplier; performs no robust extension if not provided
    

    """
  
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Print progress
    if kwargs['print_progress']: 
        print('SKU:', SKU)
    
    # Initialize results
    evaluate = Evaluation()
    results = pd.DataFrame()
    
    
    ## check if results for this SKU exist
    if os.path.exists(kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'_UPDATE.csv'):

        results = pd.read_csv(kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'_UPDATE.csv')

        # For each cost param setting
        for cost_params in kwargs['cost_params'].values():

            # Print progress
            if kwargs['print_progress']: 
                print('... cost param setting:', cost_params)

            # For each rolling look-ahead horizon
            for tau in kwargs['tau']:

                # Print progress
                if kwargs['print_progress']: 
                    print('...... look-ahead horizon:', tau)

                ## Check results over t
                res = results.loc[(results.CR == cost_params['CR']) & (results.tau == tau)]

                # If not all periods solved
                if sum(res.solutions > 0) != 13:

                    data = evaluate.prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                    y, ids_train, ids_test, weights, epsilons = data

                    # Create empty model
                    wsaamodel = RobustWeightedSAA2(**kwargs['gurobi_params'])

                    # If solved for some t
                    if sum(res.solutions > 0) > 0:

                        # Identify last solved t and the respective inventory
                        t = max(res.t[res.solutions > 0])
                        I_current = res.loc[res.t == t, 'I_q_y'].values[0]
                        print("Last solved for t = "+str(t), "with current I = "+str(I_current))

                        ids_train = ids_train[t:kwargs['T']]
                        ids_test = ids_test[t:kwargs['T']]
                        weights = weights[t:kwargs['T']]
                        epsilons = epsilons[t:kwargs['T']]

                        # Run rolling horizon model over t=1...T
                        result = evaluate.run_experiment2(y, ids_train, ids_test, tau, wsaamodel, weights=weights, 
                                                          epsilons=epsilons, I_current=I_current, **cost_params)

                        # Adjust t
                        result = result.assign(t=result.t + t)   

                    # If solved for some t
                    else:

                        t = 0
                        I_current = 0
                        print('Not solved for any t')


                    # Run rolling horizon model over t=1...T
                    result = evaluate.run_experiment2(y, ids_train, ids_test, tau, wsaamodel, weights=weights, 
                                                      epsilons=epsilons, I_current=I_current, **cost_params)

                    # Adjust t (only affects the case where at least for one t a solution existed
                    result = result.assign(t=result.t + t)   

                    # Update results
                    for col in result.columns:
                        for s in result.t:
                            results.loc[(results.CR == cost_params['CR']) & 
                                        (results.tau == tau) & 
                                        (results.t == s), col] = result.loc[result.t == s, col].values  


                            
    ## Results for this SKU do not exist
    else:
    
        # For each cost param setting
        for cost_params in kwargs['cost_params'].values():

            # Print progress
            if kwargs['print_progress']: 
                print('... cost param setting:', cost_params)

            # For each rolling look-ahead horizon
            for tau in kwargs['tau']:

                # Print progress
                if kwargs['print_progress']: 
                    print('...... look-ahead horizon:', tau)


                # Prepare data
                data = evaluate.prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                y, ids_train, ids_test, weights, epsilons = data

                # Create empty model
                wsaamodel = RobustWeightedSAA2(**kwargs['gurobi_params'])

                # Run rolling horizon model over t=1...T
                result = evaluate.run_experiment2(y, ids_train, ids_test, tau, wsaamodel, weights=weights, epsilons=epsilons, **cost_params)



                # Store result
                meta = pd.DataFrame({

                    'SKU': np.repeat(SKU,kwargs['T']),
                    'n_periods': np.repeat(kwargs['T'],kwargs['T']),
                    'tau': np.repeat(tau,kwargs['T']),
                    'CR': np.repeat(cost_params['CR'],kwargs['T']),
                    'LogToConsole': np.repeat(kwargs['gurobi_params']['LogToConsole'],kwargs['T']),
                    'Threads': np.repeat(kwargs['gurobi_params']['Threads'],kwargs['T']),
                    'NonConvex': np.repeat(kwargs['gurobi_params']['NonConvex'],kwargs['T']),
                    'PSDTol': np.repeat(kwargs['gurobi_params']['PSDTol'],kwargs['T']),
                    'MIPGap': np.repeat(kwargs['gurobi_params']['MIPGap'],kwargs['T']),
                    'obj_improvement': np.repeat(kwargs['gurobi_params']['obj_improvement'],kwargs['T']),
                    'obj_timeout_sec': np.repeat(kwargs['gurobi_params']['obj_timeout_sec'],kwargs['T']),
                    'e': np.repeat(kwargs['e'],kwargs['T']) if 'e' in kwargs else np.repeat(0,kwargs['T']),
                    'epsilon': [epsilon for epsilon in epsilons] if 'e' in kwargs else np.repeat(0,kwargs['T'])
                })

                result = pd.concat([meta, result], axis=1)

                # Store
                if not results.empty:
                    results = results.append(result)   
                else:
                    results = pd.DataFrame(result) 

   
    # Save result
    save_log = results.to_csv(
        path_or_buf=kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'_UPDATE_FINAL.csv', 
        sep=',', index=False
    )
    
    
    # Time
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Print progress
    if kwargs['print_progress']: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    
    # Returns results 
    if (kwargs['return_results'] if 'return_results' in kwargs else False):
        return results
    
    # Returns a log
    else:
        return  {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

In [3]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [4]:
# Set folder names as global variables
os.chdir('/home/fesc')
global PATH_DATA, PATH_PARAMS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Robust uncertainty set threshold multiplier
    'e': 12,

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 3*10 # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAAR',
    'model_name': 'GwSAAR',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])
       
# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments2)(SKU, **params)
                                         for SKU in SKU_range)

In [39]:
def aggregateResults(SKU_range, e, path):
    
    results = pd.DataFrame()
    
    # For all SKUs
    for SKU in SKU_range:

        # Check if results exist   
        if os.path.exists(path+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'_UPDATE_FINAL.csv'):
            results = pd.concat([results, pd.read_csv(path+'/GwSAAR_SKU'+str(SKU)+'_e'+str(e)+'_UPDATE_FINAL.csv')])

    return(results)

In [40]:
SKU_range = range(1,460+1)
e_range = [1,3,6,12]

result = {}
for e in e_range:
    result[e] = aggregateResults(SKU_range=SKU_range, e=e, path=PATH_RESULTS+'/GwSAAR')

In [41]:
Results = pd.concat([result[e] for e in e_range])

In [42]:
# Save result
Results.to_csv(path_or_buf=PATH_RESULTS+'/GwSAAR_results_UPDATE_FINAL.csv', sep=',', index=False)

In [56]:
## Get best result (best tau, best e)
GwSAAR = Results.groupby(['SKU', 'CR', 'tau', 'e']).agg({'cost': sum}).groupby(['SKU', 'CR']).agg({'cost': min}).reset_index()

In [57]:
def aggregateResults(SKU_range, path):
    
    results = pd.DataFrame()
    
    # For all SKUs
    for SKU in SKU_range:

        # Check if results exist   
        if os.path.exists(path+'/GwSAA_SKU'+str(SKU)+'.csv'):
            results = pd.concat([results, pd.read_csv(path+'/GwSAA_SKU'+str(SKU)+'.csv')])

    return(results)

In [58]:
SKU_range = range(1,460+1)
Results = aggregateResults(SKU_range=SKU_range, path=PATH_RESULTS+'/GwSAA')

In [59]:
# Save result
Results.to_csv(path_or_buf=PATH_RESULTS+'/GwSAA_results.csv', sep=',', index=False)

In [60]:
## Get best result (best tau, best e)
GwSAA = Results.groupby(['SKU', 'CR', 'tau', 'e']).agg({'cost': sum}).groupby(['SKU', 'CR']).agg({'cost': min}).reset_index()

In [92]:
## Compare
Compare = GwSAA.merge(right=GwSAAR, on=['SKU', 'CR'], suffixes=('_GwSAA', '_GwSAAR'))
Compare['costDiff'] = Compare.cost_GwSAA / Compare.cost_GwSAAR
Compare.loc[Compare.cost_GwSAA == Compare.cost_GwSAAR, 'costDiff'] = 1

In [93]:
TotalCost = Compare.groupby('CR').agg({'cost_GwSAA': sum, 'cost_GwSAAR': sum}).reset_index()
TotalCost

,CR,cost_GwSAA,cost_GwSAAR
0,0.50,7064022.0,7017212.0
1,0.75,9113644.0,9255318.5
2,0.90,12632769.5,13152278.0


In [94]:
TotalCost['costDiff'] = TotalCost.cost_GwSAA / TotalCost.cost_GwSAAR
TotalCost

,CR,cost_GwSAA,cost_GwSAAR,costDiff
0,0.50,7064022.0,7017212.0,1.006671
1,0.75,9113644.0,9255318.5,0.984693
2,0.90,12632769.5,13152278.0,0.960500


In [95]:
MedianCost = Compare.groupby('CR').agg({'costDiff': np.median}).reset_index()
MedianCost

,CR,costDiff
0,0.50,0.974819
1,0.75,0.953646
2,0.90,0.936825
